In [3]:
#!/usr/bin/env python

"""
Application:        JITC processing
File name:          jitc_datagen.py
Author:             Martin Manuel Lopez
Creation:           05/06/2024

The University of Arizona
Department of Electrical and Computer Engineering
College of Engineering
"""

# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in all
# copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
# SOFTWARE.

import os
import json
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Dense, Input
from tensorflow.keras.optimizers import Adam

class JITC_DATAOPS:
    def __init__(self, dataset):
        self.dataset = dataset
        self.data = {}  # dict to store processed data
        self.dataframe = pd.DataFrame()
        self.import_data()

    ## Step 1: Get all json files from onedrive and place them in a single directory
    ## Step 2: Run this once once you download all files and put them in a single directory
    def update_jsons(self, directory):
        for filename in os.listdir(directory):
            if filename.endswith('.json'):
                #prepend and append {} of each file
                with open(directory + '/' + filename, 'r') as f:
                    data = f.read()
                    data = '{\"binary\":' + data + '}'
                with open(directory + '/' + filename, 'w') as f:
                    f.write(data)

    ## --------------------------------------------------------------

    def change_directory(self):
        path = os.getcwd()
        ###  debug mode ---------------------------
        # testPath = str(path) + '/data/JITC_Data/'
        # os.chdir(testPath)
        #------------------------------------------
        ### run mode: change path to data directory
        path = Path(path)
        print(path)
        path = path.parents[1]
        changed_path = str(path) + '/data/JITC_Data/'
        print(changed_path)
        os.chdir(changed_path)

    # add binaries into a list
    def process_directory(self, directory):
        # print("Processing directory:", directory)
        # need to prepend each file with {} to make json file correct
        for filename in os.listdir(directory):
            if filename.endswith('.json'):
                filepath = os.path.join(directory, filename)
                self.process_json_file(filepath)

    def process_json_file(self, json_file):
        with open(json_file, 'r') as f:
            json_data = json.load(f)  # Load JSON data
            # print(json_data['binary'])
            self.data[os.path.basename(json_file)] = json_data['binary']

    def import_data(self):
        self.change_directory()
        self.process_directory(os.getcwd())
        self.dataframe = pd.DataFrame.from_dict(self.data, orient='index', columns=['binary'])
        self.dataframe.index.name = 'filename'

    def develop_dataset(self):
        # Convert binary strings to lists of integers
        self.dataframe['binary'] = self.dataframe['binary'].apply(lambda x: [int(b) for b in x])

        # Pad sequences to the same length
        sequences = pad_sequences(self.dataframe['binary'].tolist(), padding='post')

        # Assuming binary labels for the LSTM model (sum of sequence mod 2 as an example)
        labels = np.array([sum(seq) % 2 for seq in sequences])

        # Split the data into training and testing sets
        X_train, X_test, y_train, y_test = train_test_split(sequences, labels, test_size=0.2, random_state=42)

        # Reshape for LSTM [samples, time steps, features]
        X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
        X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

        return X_train, X_test, y_train, y_test

    def build_lstm_model(self, input_shape):
        inputs = Input(shape=input_shape)
        x = LSTM(50, recurrent_activation='sigmoid', recurrent_dropout=0)(inputs)
        outputs = Dense(1, activation='sigmoid')(x)

        model = Model(inputs, outputs)
        model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])
        model.summary()
        feature_extractor = Model(inputs, x)  # Model to extract features from the LSTM layer
        return model, feature_extractor

    def train_and_evaluate_model(self, model, X_train, X_test, y_train, y_test):
        history = model.fit(X_train, y_train, epochs=2, batch_size=32, validation_split=0.2) # used to be 20 epochs

        loss, accuracy = model.evaluate(X_test, y_test)
        print(f'Test Accuracy: {accuracy:.2f}')
        return history

if __name__ == "__main__":
    dataOps = JITC_DATAOPS(dataset='JITC')
    #### run the following only once to update json files
    # dataOps.update_jsons(os.getcwd() + '/data/JITC_Data')

    X_train, X_test, y_train, y_test = dataOps.develop_dataset()
    
    
    # make new directory called artifacts and change to that directory
    os.mkdir('artifacts')
    os.chdir('artifacts')
    
    # save X_train, X_test, y_train, y_test in pickle and h5 format
    X_train = pd.DataFrame(X_train.reshape(X_train.shape[0], -1))
    X_test = pd.DataFrame(X_test.reshape(X_test.shape[0], -1))
    
    print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
    
    
    X_train = pd.DataFrame(X_train)
    X_test = pd.DataFrame(X_test)
    y_train = pd.DataFrame(y_train)
    y_test = pd.DataFrame(y_test)
    X_train.to_pickle('X_train.pkl')
    X_test.to_pickle('X_test.pkl')
    y_train.to_pickle('y_train.pkl')
    y_test.to_pickle('y_test.pkl')
    

    lstm_model, feature_extractor = dataOps.build_lstm_model((X_train.shape[1], 1))
    history = dataOps.train_and_evaluate_model(lstm_model, X_train, X_test, y_train, y_test)

    # Extract features from the LSTM layer
    X_train_features = feature_extractor.predict(X_train)
    X_test_features = feature_extractor.predict(X_test)

    # os.chdir('/..')
    print(os.getcwd())
    # save features in pickle and h5 format
    X_train_features = pd.DataFrame(X_train_features)
    X_test_features = pd.DataFrame(X_test_features)
    X_train_features.to_pickle('X_train_features.pkl')
    X_test_features.to_pickle('X_test_features.pkl')
    # X_train_features.to_hdf('X_train_features.h5', key='X_train_features')
    # X_test_features.to_hdf('X_test_features.h5', key='X_test_features')


    # Train a different model (e.g., RandomForest) using the extracted features
    rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
    rf_model.fit(X_train_features, y_train)
    y_pred = rf_model.predict(X_test_features)
    rf_accuracy = accuracy_score(y_test, y_pred)
    print(f'Random Forest Test Accuracy: {rf_accuracy:.2f}')




/srv/docker/users/martinmlopez/DIS_EVL/data/testJITC
/srv/docker/users/martinmlopez/DIS_EVL/data/JITC_Data/
(4000, 241960, 1) (1000, 241960, 1) (4000,) (1000,)


Model: "functional_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 241960, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 50)             │        10,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 10,451 (40.82 KB)

 Trainable params: 10,451 (40.82 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/2
 14/100 ━━━━━━━━━━━━━━━━━━━━ 4:10:53 175s/step - accuracy: 0.4933 - loss: 0.6933